# Scipy Tutorial 2025 RAG

# **0. Prerequisites: LLM Inference Setup**
---
Before we explore the power of Retrieval-Augmented Generation, let’s first set up our LLM inference endpoint.




***Use Open Source LLM***




**Step 1: Install Required Packages**

In [ ]:
!pip install transformers accelerate huggingface-hub

**Step 2: Set Up Google Colab**



Open Google Colab, in setting Change Runtime Type, choose Runtime → Change Runtime Type to High Ram and pick a GPU









Differences: CPU vs. GPU

| Aspect            | CPU                                                         | GPU                                                      |
|-------------------|-------------------------------------------------------------|----------------------------------------------------------|
| **Function**      | Generalized component that handles main processing functions of a server | Specialized component that excels at parallel computing   |
| **Processing**    | Designed for serial instruction processing                  | Designed for parallel instruction processing             |
| **Design**        | Fewer, more powerful cores                                  | More cores than CPUs, but less powerful than CPU cores   |
| **Best suited for** | General purpose computing applications                    | High-performance computing applications                  |



Check Memory

In [10]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


Check GPU

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 16 15:41:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

**Step 3: Setup HuggingFace Token**



1.   Go to your Hugging Face account’s [Settings](https://huggingface.co/settings/tokens) → Access Tokens (huggingface.co/settings/tokens).
2.   Click “New token”, give it a name, and select the “Read” scope (sufficient for this tutorial).
3. Copy the generated token and save it in your Colab notebook as a secret in the Secrets section.

In [41]:
from google.colab import userdata
import os
from huggingface_hub import login
os.environ["HF_TOKEN"] = userdata.get("HF_Token")
login(token=os.environ["HF_TOKEN"], new_session=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


**Step 4: Instantiating a LLaMA-8B Text-Generation Pipeline with a Chat-Style Prompt**

In [22]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The pipelines are a great and easy way to use models for inference,offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

In [23]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.1,
    return_full_text=False, # don't return the prompt itself
    device_map="auto",        # spread layers across GPUs/CPU
)

Device set to use cuda:0


Integrate the LLM inference workflow into a minimal RAG helper function that lets users supply their own context.

In [25]:
def rag_generate(context: str, question: str):
    """
    context: supporting document or knowledge snippet
    question: user’s query
    """
    # build a prompt that clearly separates context from the question
    prompt = (
        f"""Context:\n{context.strip()}\n\n
        Question: {question.strip()}\n
        Answer:"""
    )
    out = pipe(prompt, max_new_tokens=500, truncation=True, do_sample=True)[0]
    return out["generated_text"]

**WITH Context**

With a clearly defined, fact-based context, the LLM can answer this question precisely.

In [37]:
context_input = """
In July 1907, Pablo Picasso unveiled “Les Demoiselles d’Avignon” in his Paris studio.
This groundbreaking canvas (243 cm × 233 cm) depicts five nude female figures with angular,
fragmented forms and faces inspired by African and Iberian masks.
By abandoning traditional single-point perspective, Picasso flattened the pictorial space
and presented multiple viewpoints simultaneously.
The painting’s radical departure from realistic representation laid the groundwork for the
Cubist movement, which Picasso and Georges Braque would develop further in 1908–1914.
"""
user_question = "What are the canvas dimensions of “Les Demoiselles d’Avignon,” and what subject does the painting depict?"

rag_generate(context_input,user_question)

' The canvas dimensions of "Les Demoiselles d’Avignon" are 243 cm × 233 cm, and the painting depicts five nude female figures with angular, fragmented forms and faces inspired by African and Iberian masks.'

**WITHOUT Context**

Without a defined knowledge context, the LLM may hallucinate and provide inaccurate information.

In [38]:
rag_generate("None",user_question)

' The canvas dimensions of "Les Demoiselles d’Avignon" are 39.37 x 29.97 inches (100 x 76 cm), and the painting depicts five young women in the streets of Avignon, Spain.'

# **References**

https://aws.amazon.com/compare/the-difference-between-gpus-cpus/